In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)

True

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7fdbca53f220>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7fdb789ffbe0>, root_client=<openai.OpenAI object at 0x7fdbca531d00>, root_async_client=<openai.AsyncOpenAI object at 0x7fdbca53f280>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [4]:
"""
In your chain, the fake_db_retrieval_step takes a question as input and pairs it with the facts from the file.
This means the facts are being injected into the pipeline at runtime, based on the question.
The retrieval step ensures the facts are fetched fresh each time the chain runs.
"""
from langsmith import traceable

@traceable(run_type="retriever")
def fake_db_retrieval_step(question):
    with open('polly_facts.txt', 'r') as file:
        polly_facts = file.read()
    result = {"question": question, "facts": polly_facts}
    print("After fake_db_retrieval_step:", result)  # 🖨️ Checkpoint 1
    return result

In [5]:
from langsmith import Client
import os
key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=key)
prompt = client.pull_prompt("polly-prompt-1:78646c84")
print("After prompt, before LLM:", prompt)  # 🖨️ Checkpoint 2

After prompt, before LLM: input_variables=['facts', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'polly-prompt-1', 'lc_hub_commit_hash': '78646c849bdc0e037bb11610953b4f3d3edd3a5d397fccbf9c331b2dc3084b77'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['facts'], input_types={}, partial_variables={}, template='You are a parrot named Polly!  Here are some facts about yourself:\n\n{facts}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]


In [6]:
"""
The prompt in your chain is probably a template that expects a dictionary with question and facts (e.g., from the retrieval step).
"Question: {question}\nFacts: {facts}\nAnswer the question based on the facts."
"""
chain = fake_db_retrieval_step | prompt | llm

question = "What do you like to eat?"
chain.invoke(question)

After fake_db_retrieval_step: {'question': 'What do you like to eat?', 'facts': '1. Polly likes animal crackers.\n2. Polly does not like goldfish.\n3. Polly likes playing soccer.\n4. Polly likes playing basketball, but is not very good because Polly does not have hands.\n'}


AIMessage(content='Polly likes animal crackers! Yum!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 76, 'total_tokens': 85, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'finish_reason': 'stop', 'logprobs': None}, id='run-3df129fb-63aa-46df-a822-cede75877518-0', usage_metadata={'input_tokens': 76, 'output_tokens': 9, 'total_tokens': 85, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
prompt = client.pull_prompt("polly-prompt-1:0b423c65")

In [8]:
chain = fake_db_retrieval_step | prompt | llm

question = "What do you like to eat?"
chain.invoke(question)

AIMessage(content="J'aime manger des biscuits pour animaux !", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 89, 'total_tokens': 98, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_3267753c5d', 'finish_reason': 'stop', 'logprobs': None}, id='run-76a3cc44-7f3c-416f-98d5-977ee7f208a6-0', usage_metadata={'input_tokens': 89, 'output_tokens': 9, 'total_tokens': 98, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})